## Notebook for making predictions with deleted constant y samples


In [61]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Reading dataset stored in other notebooks

In [62]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [63]:
X_A_train = dm.data_A.iloc[:, 2:-1]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:-1]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:-1]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

Y_A_train


0         0.00
1         0.00
2         0.00
3         0.00
4        19.36
         ...  
23219     9.02
23220     0.00
23221     0.00
23222     0.00
23223     0.00
Name: pv_measurement, Length: 23224, dtype: float64

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [64]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

In [65]:
catModel_a.fit(X_A_train,Y_A_train)
catModel_b.fit(X_B_train,Y_B_train)
catModel_c.fit(X_C_train,Y_C_train)

Learning rate set to 0.067297
0:	learn: 1245.8177147	total: 8.24ms	remaining: 8.23s
1:	learn: 1184.1122592	total: 15.4ms	remaining: 7.7s
2:	learn: 1126.7861233	total: 22.6ms	remaining: 7.5s
3:	learn: 1074.5039607	total: 28.6ms	remaining: 7.13s
4:	learn: 1025.9485387	total: 34.8ms	remaining: 6.93s
5:	learn: 980.6451839	total: 40.9ms	remaining: 6.78s
6:	learn: 940.9041303	total: 54.6ms	remaining: 7.74s
7:	learn: 903.5522763	total: 70ms	remaining: 8.68s
8:	learn: 868.8090291	total: 82.8ms	remaining: 9.11s
9:	learn: 838.0931980	total: 91ms	remaining: 9.01s
10:	learn: 809.2516491	total: 97.5ms	remaining: 8.77s
11:	learn: 783.6050462	total: 104ms	remaining: 8.52s
12:	learn: 759.1809175	total: 110ms	remaining: 8.31s
13:	learn: 737.4384392	total: 115ms	remaining: 8.13s
14:	learn: 717.4159867	total: 122ms	remaining: 8.03s
15:	learn: 699.1959899	total: 130ms	remaining: 7.97s
16:	learn: 683.7493414	total: 139ms	remaining: 8.04s
17:	learn: 668.6702165	total: 147ms	remaining: 8s
18:	learn: 654.8782

## Preds

In [66]:
pred_A = catModel_a.predict(X_A_test)
pred_B = catModel_b.predict(X_B_test)
pred_C = catModel_c.predict(X_C_test)

In [67]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("./ext_preds/catboost_donate_10.csv")

df.shape

(2160, 1)